# Auto Policy Risk model with sklearn

This notebook uses the sklearn framework to quickly build a linear regression model to calculate auto policy risk. It should run in Watson Studio using a Python 3.10 environment on Cloud Pak for Data 4.6.33. To run the notebook, replace the `CPD_PASSWORD` value with your Cloud Pak for Data admin password in the cell below.

In [ ]:
CPD_USERNAME = "admin"
CPD_PASSWORD = "PASSWORD"
EXPERIMENT_NAME = "AutoPolicyRiskFactSheet"

Get information on the current environment.

In [ ]:
import os

PROJECT_UID = os.environ['PROJECT_ID']
CPD_URL = os.environ['RUNTIME_ENV_APSX_URL']
CONTAINER_ID = PROJECT_UID
CONTAINER_TYPE = "project"

WML_CREDENTIALS = {
    "url": CPD_URL,
    "password": CPD_PASSWORD,
    "username": CPD_USERNAME,
    "instance_id": "wml_local",
    "version": "4.5"
}

Get an instance of the Watson Machine Learning Client.

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.version)

Read in the training data from the project as a pandas dataframe.

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

import pandas as pd

df_data_1 = pd.read_csv(wslib.mount.get_data_path('policy_risk_training.csv'))
df_data_1.head()


Drop the unnecessary fields from the training data.

In [ ]:
df_data_1.drop(['HOUSEHOLD_ID', 'POLICY_ID', 'LAST_NAME', 'ZIPCODE'], axis=1, inplace=True, errors='ignore')

Import the necessary sklearn libraries.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

Split the train and test data.

In [ ]:
X = df_data_1.drop(['RISK'], axis=1)
y = df_data_1['RISK']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15) # 85% training and 15% test

Identify the categorical features, and set up a one hot encoder transform as part of the model pipeline. Fit the training data.

In [ ]:
categorical_features = ['PRIM_DRIVER_GENDER', 'MAKE_MODEL']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features)
    ]
)

pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearRegression())]
)

model = pipeline.fit(X_train, y_train)

Use the model to predict the test data.

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

Calculate the Root Mean Squared Error of the predictions.

In [ ]:
from math import sqrt
meanSquaredError = mean_squared_error(y_test, y_pred)
rootMeanSquaredError = sqrt(meanSquaredError)
print("RMSE:", rootMeanSquaredError)

Set the default project to the current project and prepare to store the model.

In [ ]:
wml_client.set.default_project(PROJECT_UID)

Define the model runtime and other metadata.

In [ ]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.2-py3.10")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
    wml_client._models.ConfigurationMetaNames.NAME:"{}".format("auto policy risk - sklearn"),
    wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.1",
    wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "RISK"
}


Store the model in the current project.

In [ ]:
print("Storing model ...")
published_model_details = wml_client.repository.store_model(
    model=model, 
    meta_props=model_props, 
    training_target=['Risk'],
    training_data=X_train,
    pipeline=pipeline)

model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))